<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [ ]:
# Write your code here
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [ ]:
!pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

chrome_driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\maganti\AppData\Local\Temp\ipykernel_27164\2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

#driver = webdriver.Chrome(executable_path="C:\chromedriver_win32",options=options)
link = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
title_array = []
review_array = []
chrome_driver.get(link)

for num in range(4):
  chrome_driver.find_element(By.CLASS_NAME, "ipl-load-more__button").click()
  time.sleep(5)
  listOfTitle = chrome_driver.find_elements(By.CLASS_NAME, "title")
  listOfReviews = chrome_driver.find_elements(By.CLASS_NAME, "text")
    
for ele, sub_ele in zip(listOfTitle, listOfReviews):
      title_array.append((ele.text).replace('\n',''))
      review_array.append(sub_ele.text)
    
df = pd.DataFrame(list(zip(title_array, review_array)), columns =['Title', 'Review'])
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 124


,Title,Review
0,Perfect in every aspect.,"Truly a masterpiece, The Best Hollywood film o..."
1,"A psychological study, rather than a superhero...",I have seen Joker yesterday at Venice an early...
2,Went for a second time to watch,I get why some people hate this . It's because...
3,JUST AMAZING. How does this movie exist.,Let me start off by saying if Joaquin Phoneix ...
4,Outstanding movie with a haunting performance ...,"Every once in a while a movie comes, that trul..."
...,...,...
119,Masterpiece *Movie Of the Year* Best Actor - J...,"Wow I honestly gotta tell you, it's one of the..."
120,The Joker is supposed to be a diabolical maste...,
121,Better if didn't saw,Disturbing. Just got out of the movie. My humo...
122,Why Todd Phillipps Made This Film,You will notice the far left journalists: the ...


In [ ]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [ ]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

words_sentence = []
for sentence in df['Review']:
    words_sentence.append(word_tokenize(sentence))
aftercleaning = [x for x in words_sentence if x != []]
iterations = list(itertools.chain.from_iterable(aftercleaning))

In [ ]:
trigrams = nltk.trigrams(iterations)
frequency_dist = nltk.FreqDist(trigrams)
frequency_dist

FreqDist({('.', 'It', "'s"): 22, (',', 'it', "'s"): 19, ('.', 'This', 'is'): 14, (',', 'but', 'it'): 11, ('.', 'This', 'movie'): 11, ('one', 'of', 'the'): 10, ('it', "'s", 'a'): 10, ('.', 'It', 'is'): 10, ('this', 'movie', ','): 10, ('I', 'do', "n't"): 10, ...})

In [ ]:
from collections import Counter

bigrams = nltk.bigrams(iterations)
frequency_dist = nltk.FreqDist(bigrams)
bigrams_dict = dict(frequency_dist)
count = 0

for word in bigrams_dict:
    if count > 10:
        break
    print( str(word) + ':' + str(bigrams_dict[word] / iterations.count(word[0])))
    count=count+1

('Truly', 'a'):1.0
('a', 'masterpiece'):0.01702127659574468
('masterpiece', ','):0.25
(',', 'The'):0.004310344827586207
('The', 'Best'):0.009615384615384616
('Best', 'Hollywood'):0.16666666666666666
('Hollywood', 'film'):0.16666666666666666
('film', 'of'):0.02608695652173913
('of', '2019'):0.013812154696132596
('2019', ','):0.125
(',', 'one'):0.0028735632183908046


In [ ]:
whole_dataset = ''
index = []
i = 1
for line in df['Review']:
    whole_dataset = whole_dataset + line
    value = 'Review-' + str(i)
    index.append(value)
    i += 1

In [ ]:
from textblob import TextBlob
noun_phrases = []
frequency = []

for line in df['Review']:
    blob = TextBlob(line)
    for nouns in blob.noun_phrases:
        noun_phrases.append(nouns)
    
    
for word in noun_phrases:
    noun_phrases_freq = []
    for line in df['Review']:
        try:
            noun_phrases_freq.append(line.count(word) / whole_dataset.count(word))
        except:
            pass
    frequency.append(noun_phrases_freq)
    
noun_phrases_df = pd.DataFrame(frequency).T
noun_phrases_df.columns = list(noun_phrases)
noun_phrases_df.index = index
noun_phrases_df.head(5)

,truly,hollywood,best,decade ...,best,comic book,real ife,direction,cinematography,disturbing,...,oscars,joker,joker,oscars,upon,time ...,hollywood,joker,oscar,joker
Review-1,0.1,NaN,0.000000,NaN,0.000000,0.076923,1.0,0.0,0.000000,0.000000,...,0.0,0.000,0.000,0.0,0.0,NaN,NaN,0.000,0.0,0.000
Review-2,0.0,NaN,0.038462,NaN,0.038462,0.000000,0.0,0.0,0.058824,0.000000,...,1.0,0.125,0.125,1.0,0.0,NaN,NaN,0.125,0.5,0.125
Review-3,0.0,NaN,0.000000,NaN,0.000000,0.000000,0.0,0.0,0.058824,0.000000,...,0.0,0.000,0.000,0.0,0.0,NaN,NaN,0.000,0.0,0.000
Review-4,0.0,NaN,0.038462,NaN,0.038462,0.000000,0.0,0.0,0.000000,0.166667,...,0.0,0.000,0.000,0.0,0.0,NaN,NaN,0.000,0.0,0.000
Review-5,0.1,NaN,0.038462,NaN,0.038462,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000,0.000,0.0,0.0,NaN,NaN,0.000,0.0,0.000


# **Question 2: Undersand TF-IDF and Document representation**

(20 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [ ]:
# Write your code here

import math
def tf_idf_value(sentence,word):
  size = len(sentence.split(" "))
  tf_value = sentence.count(word)/size
  idf_value = 0
  if(tf_value!=0):
    idf_value = math.log(size)/sentence.count(word)
  else:
    return 0;
  return tf_value*idf_value

sentences = df["Review"].values.tolist()
tokens_list = set([j for i in sentences for j in i.split(" ")])
tf_idf = pd.DataFrame(tokens_list,columns=["token"])
count = 0
for i in sentences:
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
  count+=1

tf_idf.head(5)





C:\Users\maganti\AppData\Local\Temp\ipykernel_27164\3804204924.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
C:\Users\maganti\AppData\Local\Temp\ipykernel_27164\3804204924.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
C:\Users\maganti\AppData\Local\Temp\ipykernel_27164\3804204924.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

C:\Users\maganti\AppData\Local\Temp\ipykernel_27164\3804204924.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))
C:\Users\maganti\AppData\Local\Temp\ipykernel_27164\3804204924.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tf_idf[str(count)] = tf_idf["token"].apply(lambda x : tf_idf_value(i,x))


,token,0,1,2,3,4,5,6,7,8,...,114,115,116,117,118,119,120,121,122,123
0,,0.035297,0.020065,0.060693,0.033765,0.053239,0.054252,0.0,0.044658,0.035912,...,0.009926,0.04043,0.044997,0.075985,0.013377,0.018654,0.0,0.092222,0.019776,0.024792
1,sloppily,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,medal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.024792
3,role,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.024792
4,"superficial,",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


# **Question 3: Create your own word embedding model**

(20 points). Use the data you collected for assignment two to build a word embedding model: 

(1) Train a 300-dimension word embedding (it can be word2vec, glove, ulmfit, bert, or others).

(2) Visualize the word embedding model you created.

Reference: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

Reference: https://jaketae.github.io/study/word2vec/

In [ ]:
# Write your code here

from gensim.models import Word2Vec

In [ ]:
sentences = [['text', 'images', 'videos', 'can', 'be', 'summarized'],
             ['video', 'summarization', 'extracts', 'most', 'important', 'video', 'frames'],
            ['content', 'is', 'extracted', 'from', 'original', 'data'],
            ['abstractive ', 'summarization', 'generates', 'new', 'text', 'that', 'did', 'not', 'exist', 'in', 'the', 'original', 'text']]

model = Word2Vec(sentences, min_count=1)

words = list(model.wv.index_to_key)
print(words)
print(model.wv['summarization'])



['text', 'video', 'summarization', 'original', 'content', 'images', 'videos', 'can', 'be', 'summarized', 'extracts', 'most', 'important', 'frames', 'the', 'in', 'extracted', 'from', 'data', 'abstractive ', 'generates', 'new', 'that', 'did', 'not', 'exist', 'is']
[ 9.3063500e-05  3.0791333e-03 -6.8122302e-03 -1.3768083e-03
  7.6687024e-03  7.3447735e-03 -3.6728731e-03  2.6438569e-03
 -8.3157662e-03  6.2035224e-03 -4.6357326e-03 -3.1641286e-03
  9.3115270e-03  8.7194168e-04  7.4924259e-03 -6.0736607e-03
  5.1627443e-03  9.9216020e-03 -8.4582577e-03 -5.1349136e-03
 -7.0642880e-03 -4.8619211e-03 -3.7774493e-03 -8.5359123e-03
  7.9526650e-03 -4.8434949e-03  8.4246937e-03  5.2631958e-03
 -6.5489644e-03  3.9572939e-03  5.4698125e-03 -7.4271183e-03
 -7.4075484e-03 -2.4749818e-03 -8.6261127e-03 -1.5804928e-03
 -4.0058116e-04  3.2991031e-03  1.4399484e-03 -8.7803742e-04
 -5.5922056e-03  1.7317130e-03 -8.9541968e-04  6.7925104e-03
  3.9753579e-03  4.5302045e-03  1.4344518e-03 -2.7004278e-03
 -4.3

# **Question 4: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [1]:
# The GitHub link of your final csv file



# Link: https://github.com/sowmyapopurii/popuri_5731/blob/main/Assignment%203.xlsx



